In [28]:
import os
import random
import jieba
import jieba.analyse


def TextProcessing(words):
    # word_cut = jieba.cut(words, cut_all=False)  # 精简模式，返回一个可迭代的generator
    # word_list = list(word_cut)  # generator转换为list
    # return word_list
    # seg = jieba.cut_for_search(words)  # 搜索引擎模式
    # seg_list = list(seg)
    #
    # print(", ".join(seg_list))
    # return seg_list
    keywords = jieba.analyse.extract_tags(words, topK=20, withWeight=True, allowPOS=('n', 'nr', 'ns', 'nz', 'd', 'nt', 'v', 'vn', 'i'))
    # print(keywords)
    # print(type(keywords))
    # <class 'list'>
    item_list = []
    for item in keywords:
        # print(item)
        # print(item[0], item[1])
        all_item = {}

        all_item['words'] = item[0]
        all_item['weight'] = item[1]
        item_list.append(all_item)

    return item_list


def MakeWordsSet(words_file):
    words_set = set()                                            # 创建set集合
    with open(words_file, 'r', encoding='utf-8') as f:        # 打开文件
        for line in f.readlines():                                # 一行一行读取
            word = line.strip()                                    # 去回车
            if len(word) > 0:                                    # 有文本，则添加到words_set中
                words_set.add(word)
    return words_set                                             # 返回处理结果


def words_dict(all_words_list,deleteN,stopwords_set=set()):
    feature_words = []   # 特征列表
    n = 1
    for t in range(deleteN, len(all_words_list), 1):
        if n > 1000:  # feature_words的维度为1000
            break
        # 如果这个词不是数字，并且不是指定的结束语，并且单词长度大于1小于5，那么这个词就可以作为特征词
        if not all_words_list[t]['words'].isdigit() and all_words_list[t]['words'] not in stopwords_set and 1 < len(all_words_list[t]['words']) < 5:
            feature_words.append(all_words_list[t])
        n += 1
    # print(feature_words)
    return feature_words


if __name__ == '__main__':
    # 文本预处理
    # folder_path = './SogouC/Sample'                #训练集存放地址
    # all_words_list, train_data_list, test_data_list, train_class_list, test_class_list = TextProcessing(folder_path, test_size=0.2)
    
    # 导入数据(一小部分的新闻数据)
    df_news = pd.read_table('ceshi_con.csv', names=['content'], encoding='utf-8')
    df_news = df_news.dropna()  # 直接丢弃包括NAN的整条数据
    print(df_news.head())

#     # 分词
#     content = df_news.content.values.tolist()  # 因为jieba要列表格式
#     content_S = []  # 存储分完词之后结果
#     for line in content:
#         current_segment = jieba.lcut(line)  # jieba分词
#         if len(current_segment) > 1 and current_segment != "\r\n":
#             content_S.append(current_segment)
#     # 将分完词的结果转化成DataFrame格式
#     df_content = pd.DataFrame({"content_S": content_S})
#     print("---------")
#     print(df_content.head())

     
#     all_words_list = TextProcessing('新浪新闻-“断交”民进党真急了 恼羞成怒喊“换不来统一”')
    for line in df_news:
        all_words_list = TextProcessing('line')
    

    # 生成stopwords_set
        stopwords_file = 'stopwords.txt'
        stopwords_set = MakeWordsSet(stopwords_file)

        feature_words = words_dict(all_words_list, 2, stopwords_set)
        print(feature_words)

                                             content
0  242624沈阳抢那么多人干啥来源：真话财经（ID:zhenhuacaijing）作者：...
1  243456东南亚人和中国人的不同，是好还是坏？来源 | 公众号 墨腾创投作者 | 墨腾...
2  244192AI“造人”VS “无人”？解放双手的盛宴或是无用阶级的丧钟“造人”VS“无...
3  245760我们一直在说扩张的城市、增长的经济，那我们的收缩城市呢？那天晚上我就问自己是...
4  243008《创造101》：一场时代共鸣下的“温柔爆发”“请给我舞台，成败都可爱。” 春...
[]


In [3]:
import pandas as pd
import jieba
import jieba.analyse
import kashgari
from collections import Counter


# 将文章分词再利用自己设定的stopwords来进行清洗,最后将清洗后的分词来进行整合
# 数据(一小部分的新闻数据)
# 去除停用词函数
def drop_stopwords(contents, stopwords):
    contents_clean = []
    all_words = []
    for line in contents:
        line_clean = []
        for word in line:
            if word in stopwords:
                continue
            line_clean.append(word)
            all_words.append(str(word))  # 所有的词组成一个列表
        contents_clean.append(line_clean)
    return contents_clean, all_words

# 导入数据(一小部分的新闻数据)
df_news = pd.read_table('aid_title_content.txt', names=['content'], encoding='utf-8')
df_news = df_news.dropna()  # 直接丢弃包括NAN的整条数据
# print(df_news)

# 分词
content = df_news.content.values.tolist()  # 因为jieba要列表格式
content_S = []  # 存储分完词之后结果
for line in content:
    current_segment = jieba.lcut(line)  # jieba分词
    if len(current_segment) > 1 and current_segment != "\r\n":
        content_S.append(current_segment)
# 将分完词的结果转化成DataFrame格式
df_content = pd.DataFrame({"content_S": content_S})
# print("-----")
# print(df_content)

# 清洗乱的数据，用停用词表去除停用词
stopwords = pd.read_csv('stopwords.txt', index_col=False, sep='\t', quoting=3, names=["stopword"],
                        encoding="utf-8")  # 读入停用词

# 调用去除停用词函数
contents = df_content.content_S.values.tolist()
stopwords = stopwords.stopword.values.tolist()
contents_clean, all_words = drop_stopwords(contents, stopwords)

# 将清洗完的数据结果转化成DataFrame格式
# print(df_content)
df_content = pd.DataFrame({"contents_clean": contents_clean})

# a来储存清洗后的文章内容
a = []
for line in df_content['contents_clean']:
    # line = line.values.tolist()
    a.append("".join(line))





'''
利用tdidf来进行关键词的提取
'''
df_news=a
# 分词上面已经转化成列表格式了直接用
content = a# 因为jieba要列表格式
content_S = []  # 存储分完词之后结果
for line in content:
    current_segment = jieba.lcut(line)  # jieba分词
    if len(current_segment) > 1 and current_segment != "\r\n":
        content_S.append(current_segment)
# 将分完词的结果转化成DataFrame格式
df_content = pd.DataFrame({"content_S": content_S})

# tdidf
list1 = []
for line in content:
    title=line[0:6]
    content=line
    top_n=100
    # print(content)
    # print(title)
    # print(top_n)
    keywords = jieba.analyse.extract_tags(line, topK=10, withWeight=True, allowPOS=('ns', 'n'))
    list = []
    for i in range(len(keywords)):
        # print(len(keywords))
        # print(keywords[i][0])
        # print(keywords[i][1])
        # dict = {'tfidf': keywords[i][1],  'word': keywords[i][0]}
        dict = keywords[i][0]
        # print(dict)
        list.append(dict)
    list1.append(list)
# print(list1)
a = list1
# print(type(a))
a = a
# 方法二，列表推导式：
newlist = [i for j in range(len(a)) for i in a[j]]
# print(newlist)
def getUniqueItems(iterable):
    seen = set()
    result = []
    for item in iterable:
        if item not in seen:
            seen.add(item)
            result.append(item)
    return result
a = newlist
# a = ["asd", "def", "ase", "dfg", "asd", "def", "dfg"]
# print(type(a))
print(getUniqueItems(a))

['电商', '医药', '药品', '用户', '互联网', '处方药', '药物', '国药', '物流', '买药', '京东', '冷链', '果园', '水果', '系统', '产品品质', '市场', '创业者', '创业', '奇点', '大赛', '媒体', '东楼', '公司', '大学', '项目', '网贷', '平台', '投资者', '借贷', '机构', '趣谈', '行业', '优点', '星火', '百视通', '业务', '电视', '内容', '集团', '团队', '入口', '美国', '中国', '事情', '佛家', '哲学', '竞争者', '道家', '商场', '金融', '信用', '大学生', '数据', '海量', '利融', '芝麻', '智能', '硬件', '虚拟现实', '手机', '手表', '游戏机', '解决方案', '产品', '场景', '领域', '眼镜', '画面', '优势', '建模', '事物', '小米', '体验', '账号', '厂商', '语言', '品牌', '百度', '太子', '接班人', '块钱', '如雾', '钦点', '浏览器', '钱包', '计划', '影城', '屏幕', '全球', '电影', '净利润', '票房', '租金', '目标', '蚂蚁', '人类', '程序', '模式', '流水', '路径', '能力', '膝下', '资本', '运营商', '电信', '平均收入', '广义', '欧美', '话单', '流失率', '通话记录', '用户服务', '导致用户', '股票', '业绩', '三板', '营收', '企业', '概念股', '交易日', '股价', '移民', '禁令', '加州', '白人', '族裔', '投票', '民调', '全球化', '版权', '赛事', '苏宁', '独家', '俱乐部', '转播权', '足球', '代理', '游戏', '玩法', '风险', '工委', '版号', '玩家', '手游', '法律', '小虫', '骆驼', '成衣', '时尚', '设计师', '女性', '眼光', '流量', '方式', '省份', '套餐', '悲剧', '思维', '现场', '传统', 